In [172]:
import pandas as pd

prolongations = pd.read_csv('prolongations.csv')
financial_data = pd.read_csv('financial_data.csv')

financial_data.replace('\xa0', '', regex=True, inplace=True)
financial_data.replace(',', '.', regex=True, inplace=True)

In [173]:
managers = prolongations["AM"].unique()

In [174]:
all_months = ['ноябрь 2022', 'декабрь 2022', 'январь 2023', 'февраль 2023',
              'март 2023', 'апрель 2023', 'май 2023', 'июнь 2023', 'июль 2023',
              'август 2023', 'сентябрь 2023', 'октябрь 2023', 'ноябрь 2023',
              'декабрь 2023', 'январь 2024', 'февраль 2024']

months_to_calc_prolongation = ['январь 2023', 'февраль 2023', 'март 2023', 'апрель 2023',
                  'май 2023', 'июнь 2023', 'июль 2023', 'август 2023',
                  'сентябрь 2023', 'октябрь 2023', 'ноябрь 2023', 'декабрь 2023']

In [175]:
# для начала посчитаем коэффициенты пролонгации для каждого менеджера:
# 1) за каждый месяц (2023 год)
# 2) за год (2023 год)

In [176]:
def capitalize_first_letter(string):
    return string[0].upper() + string[1:]

def replace_zeros_with_previous(df):
    for index, row in df.iterrows():
        previous_value = None
        for column in df.columns[1:]:
            if row[column] != 0:
                previous_value = row[column]
            elif row[column] == 0 and previous_value is not None:
                df.at[index, column] = previous_value
    return df

def remove_rows_with_zeros(df):
    return df[~df.apply(lambda row: (row == 0).any(), axis=1)]


In [177]:
import numpy as np

for manager in managers:
    for prolongation_month in months_to_calc_prolongation:
        # получаем все проекты, которые были завершены в предыдущих месяцах
        prolongation_month_index = all_months.index(prolongation_month)
        month0_index = all_months.index(prolongation_month) - 2
        month1_index = all_months.index(prolongation_month) - 1
        month0 = all_months[month0_index]
        month1 = all_months[month1_index]

        manager_completed_projects = prolongations[prolongations['AM'] == manager]
        
        month0_completed_projects = manager_completed_projects[manager_completed_projects['month'] == month0]
        month1_completed_projects = manager_completed_projects[manager_completed_projects['month'] == month1]
        
        k2_up_sum = 0
        k2_down_sum = 0
        
        ids = month0_completed_projects['id'].unique()
        projects_financial_data = financial_data[financial_data['id'].isin(ids)].copy()
        
        for id, project in projects_financial_data.iterrows():
            for check_month in range(0, prolongation_month_index + 1):
                if project[capitalize_first_letter(all_months[check_month])] == "стоп" or project[capitalize_first_letter(all_months[check_month])] == "end":
                    projects_financial_data.drop(id, inplace=True)
        
        required_months = all_months[0: prolongation_month_index + 1]
        required_months = [capitalize_first_letter(month) for month in required_months]
        required_months = ['id'] + required_months

        projects_financial_data = projects_financial_data[required_months]
        print("До:")
        print(projects_financial_data)

        projects_financial_data = projects_financial_data.replace("в ноль", 0)
        projects_financial_data = projects_financial_data.replace("NaN", np.nan)
        projects_financial_data = projects_financial_data.apply(pd.to_numeric, errors='coerce')

        print("После:")
        print(projects_financial_data)
        
        projects_financial_data_grouped = projects_financial_data.groupby('id').sum(min_count=1)
        print("Группировка:")
        print(projects_financial_data_grouped)

        projects_financial_data_grouped = replace_zeros_with_previous(projects_financial_data_grouped)
        print("После замены нулей:")
        print(projects_financial_data_grouped)
        
        projects_financial_data_grouped = remove_rows_with_zeros(projects_financial_data_grouped)
        print("После удаления нулей:")
        print(projects_financial_data_grouped)
        
        
# # Применяем функцию для каждой группы 'id'
# processed_data = df.groupby('id').apply(process_group)
# 
# # Заменяем 'В ноль' на предыдущие значения в строках
# def replace_zeroes_with_previous(df):
#     for column in df.columns:
#         df[column] = df[column].replace('В ноль', np.nan)
#         df[column] = df[column].fillna(method='ffill')  # Заполняем предыдущим значением
# 
#     # Удаляем строки с NaN (если они остались после замены)
#     df = df.dropna()
#     return df
# 
# processed_data = replace_zeroes_with_previous(processed_data)
# 
# # Печать финального результата
# print(processed_data)
#         # print(projects_financial_data_grouped)
        

        # print("проекты без стопов:")
        # print(projects_financial_data)
        
            # if float(project[capitalize_first_letter(prev_prev_month)]):
            #     k2_up_sum += float(project[capitalize_first_letter(prev_prev_month)])
            #     k2_down_sum += float(project[capitalize_first_letter(prev_month)])
            #     print(f"Проект {project['id']} в месяце {prev_prev_month} имеет значение {project[capitalize_first_letter(prev_prev_month)]}")
        # for id in prev_prev_month_completed_projects['id']:
        #     projects = financial_data.loc[financial_data['id'] == id]
        #     for check_month in range(0, prev_prev_month_index + 1):
        #         disbursement = projects[capitalize_first_letter(all_months[check_month])]
        #         print(len(disbursement))
        #         print(disbursement)
            # for _, project in projects.iterrows():
            #     if float(project[capitalize_first_letter(prev_month)]):
            #         print(f"Проект {id} в месяце {prev_month} имеет значение {project[capitalize_first_letter(prev_month)]}")
                
# for months in months_to_calc:
#     prev_prev_month = all_months[all_months.index(months) - 2]
#     prev_month = all_months[all_months.index(months) - 1]
# 
#     prev_prev_month_completed_projects = prolongations[prolongations['month'] == prev_prev_month]
#     prev_month_completed_projects = prolongations[prolongations['month'] == prev_month]
    

До:
      id Ноябрь 2022 Декабрь 2022 Январь 2023
0     42    36220.00          NaN         NaN
4    665    10000.00          NaN         NaN
8    579      в ноль          NaN         NaN
9    592    92302.00          NaN         NaN
10   592    12340.00          NaN         NaN
11   666   110000.00          NaN         NaN
12   620      в ноль          NaN         NaN
83   453      в ноль     39245.00    44320.00
160  600   237977.00    144650.00   144650.00
161  600    43440.00     30450.00   122570.00
217  633    40200.00     40200.00    40200.00
311  369    42600.00      7450.00         NaN
312  369         NaN          NaN         NaN
После:
      id  Ноябрь 2022  Декабрь 2022  Январь 2023
0     42      36220.0           NaN          NaN
4    665      10000.0           NaN          NaN
8    579          0.0           NaN          NaN
9    592      92302.0           NaN          NaN
10   592      12340.0           NaN          NaN
11   666     110000.0           NaN          NaN
12